In [49]:
# look at tools/set_up_magics.ipynb
yandex_metrica_allowed = True ; get_ipython().run_cell('# one_liner_str\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \'// setup cpp code highlighting\\nIPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\')\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE\nimport random\nimport sys\nimport os\nimport re\nimport signal\nimport shutil\nimport shlex\nimport glob\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                if line.startswith(run_prefix):\n                    cmds.append(line[len(run_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                run_prefix = "%# "\n                if line.startswith(run_prefix):\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                raise Exception("Unknown %%save_file subcommand: \'%s\'" % line)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        display(Markdown("Run: `%s`" % cmd))\n        get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    assert not fname\n    save_file("makefile", cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    try:\n        expr, comment = line.split(" #")\n        display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n    except:\n        display(Markdown("{} = {}".format(line, eval(line))))\n        \ndef show_file(file, clear_at_begin=True, return_html_string=False):\n    if clear_at_begin:\n        get_ipython().system("truncate --size 0 " + file)\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                elem.innerText = xmlhttp.responseText;\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (errors___OBJ__ < 10 && !entrance___OBJ__) {\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n        \n        <font color="white"> <tt>\n        <p id="__OBJ__" style="font-size: 16px; border:3px #333333 solid; background: #333333; border-radius: 10px; padding: 10px;  "></p>\n        </tt> </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__ -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n    \nBASH_POPEN_TMP_DIR = "./bash_popen_tmp"\n    \ndef bash_popen_terminate_all():\n    for p in globals().get("bash_popen_list", []):\n        print("Terminate pid=" + str(p.pid), file=sys.stderr)\n        p.terminate()\n    globals()["bash_popen_list"] = []\n    if os.path.exists(BASH_POPEN_TMP_DIR):\n        shutil.rmtree(BASH_POPEN_TMP_DIR)\n\nbash_popen_terminate_all()  \n\ndef bash_popen(cmd):\n    if not os.path.exists(BASH_POPEN_TMP_DIR):\n        os.mkdir(BASH_POPEN_TMP_DIR)\n    h = os.path.join(BASH_POPEN_TMP_DIR, str(random.randint(0, 1e18)))\n    stdout_file = h + ".out.html"\n    stderr_file = h + ".err.html"\n    run_log_file = h + ".fin.html"\n    \n    stdout = open(stdout_file, "wb")\n    stdout = open(stderr_file, "wb")\n    \n    html = """\n    <table width="100%">\n    <colgroup>\n       <col span="1" style="width: 70px;">\n       <col span="1">\n    </colgroup>    \n    <tbody>\n      <tr> <td><b>STDOUT</b></td> <td> {stdout} </td> </tr>\n      <tr> <td><b>STDERR</b></td> <td> {stderr} </td> </tr>\n      <tr> <td><b>RUN LOG</b></td> <td> {run_log} </td> </tr>\n    </tbody>\n    </table>\n    """.format(\n        stdout=show_file(stdout_file, return_html_string=True),\n        stderr=show_file(stderr_file, return_html_string=True),\n        run_log=show_file(run_log_file, return_html_string=True),\n    )\n    \n    cmd = """\n        bash -c {cmd} &\n        pid=$!\n        echo "Process started! pid=${{pid}}" > {run_log_file}\n        wait ${{pid}}\n        echo "Process finished! exit_code=$?" >> {run_log_file}\n    """.format(cmd=shlex.quote(cmd), run_log_file=run_log_file)\n    # print(cmd)\n    display(HTML(html))\n    \n    p = Popen(["bash", "-c", cmd], stdin=PIPE, stdout=stdout, stderr=stdout)\n    \n    bash_popen_list.append(p)\n    return p\n\n\n@register_line_magic\ndef bash_async(line):\n    bash_popen(line)\n    \n    \ndef show_log_file(file, return_html_string=False):\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function halt__OBJ__(elem, color)\n        {\n            elem.setAttribute("style", "font-size: 14px; background: " + color + "; padding: 10px; border: 3px; border-radius: 5px; color: white; ");                    \n        }\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            if (entrance___OBJ__ < 0) {\n                entrance___OBJ__ = 0;\n            }\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                if (elem.innerHTML != xmlhttp.responseText) {\n                                    elem.innerHTML = xmlhttp.responseText;\n                                }\n                                if (elem.innerHTML.includes("Process finished.")) {\n                                    halt__OBJ__(elem, "#333333");\n                                } else {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                }\n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (!entrance___OBJ__) {\n                                if (errors___OBJ__ < 6) {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                } else {\n                                    halt__OBJ__(elem, "#994444");\n                                }\n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n\n        <p id="__OBJ__" style="font-size: 14px; background: #000000; padding: 10px; border: 3px; border-radius: 5px; color: white; ">\n        </p>\n        \n        </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__.md -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n\n    \nclass TInteractiveLauncher:\n    tmp_path = "./interactive_launcher_tmp"\n    def __init__(self, cmd):\n        try:\n            os.mkdir(TInteractiveLauncher.tmp_path)\n        except:\n            pass\n        name = str(random.randint(0, 1e18))\n        self.inq_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".inq")\n        self.log_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".log")\n        \n        os.mkfifo(self.inq_path)\n        open(self.log_path, \'w\').close()\n        open(self.log_path + ".md", \'w\').close()\n\n        self.pid = os.fork()\n        if self.pid == -1:\n            print("Error")\n        if self.pid == 0:\n            exe_cands = glob.glob("../tools/launcher.py") + glob.glob("../../tools/launcher.py")\n            assert(len(exe_cands) == 1)\n            assert(os.execvp("python3", ["python3", exe_cands[0], "-l", self.log_path, "-i", self.inq_path, "-c", cmd]) == 0)\n        self.inq_f = open(self.inq_path, "w")\n        interactive_launcher_opened_set.add(self.pid)\n        show_log_file(self.log_path)\n\n    def write(self, s):\n        s = s.encode()\n        assert len(s) == os.write(self.inq_f.fileno(), s)\n        \n    def get_pid(self):\n        n = 100\n        for i in range(n):\n            try:\n                return int(re.findall(r"PID = (\\d+)", open(self.log_path).readline())[0])\n            except:\n                if i + 1 == n:\n                    raise\n                time.sleep(0.1)\n        \n    def input_queue_path(self):\n        return self.inq_path\n        \n    def close(self):\n        self.inq_f.close()\n        os.waitpid(self.pid, 0)\n        os.remove(self.inq_path)\n        # os.remove(self.log_path)\n        self.inq_path = None\n        self.log_path = None \n        interactive_launcher_opened_set.remove(self.pid)\n        self.pid = None\n        \n    @staticmethod\n    def terminate_all():\n        if "interactive_launcher_opened_set" not in globals():\n            globals()["interactive_launcher_opened_set"] = set()\n        global interactive_launcher_opened_set\n        for pid in interactive_launcher_opened_set:\n            print("Terminate pid=" + str(pid), file=sys.stderr)\n            os.kill(pid, signal.SIGKILL)\n            os.waitpid(pid, 0)\n        interactive_launcher_opened_set = set()\n        if os.path.exists(TInteractiveLauncher.tmp_path):\n            shutil.rmtree(TInteractiveLauncher.tmp_path)\n    \nTInteractiveLauncher.terminate_all()\n   \nyandex_metrica_allowed = bool(globals().get("yandex_metrica_allowed", False))\nif yandex_metrica_allowed:\n    display(HTML(\'\'\'<!-- YANDEX_METRICA_BEGIN -->\n    <script type="text/javascript" >\n       (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};\n       m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})\n       (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");\n\n       ym(59260609, "init", {\n            clickmap:true,\n            trackLinks:true,\n            accurateTrackBounce:true\n       });\n    </script>\n    <noscript><div><img src="https://mc.yandex.ru/watch/59260609" style="position:absolute; left:-9999px;" alt="" /></div></noscript>\n    <!-- YANDEX_METRICA_END -->\'\'\'))\n\ndef make_oneliner():\n    html_text = \'("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "")\'\n    html_text += \' + "<""!-- MAGICS_SETUP_PRINTING_END -->"\'\n    return \'\'.join([\n        \'# look at tools/set_up_magics.ipynb\\n\',\n        \'yandex_metrica_allowed = True ; get_ipython().run_cell(%s);\' % repr(one_liner_str),\n        \'display(HTML(%s))\' % html_text,\n        \' #\'\'MAGICS_SETUP_END\'\n    ])\n       \n\n');display(HTML(("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "") + "<""!-- MAGICS_SETUP_PRINTING_END -->")) #MAGICS_SETUP_END

<IPython.core.display.Javascript object>

Run: `clang++ -Wall -Werror main.cpp -o a.exe`

Run: `time -p ./a.exe`

real 0.34
user 0.24
sys 0.09


Run: `clang++ -Wall -Werror main.cpp -o a.exe`

Run: `time -p ./a.exe`

real 0.28
user 0.23
sys 0.04


In [73]:
%%cpp cb.h

#ifndef DEQUE_CIRCULAR_BUFFER_H
#define DEQUE_CIRCULAR_BUFFER_H

#include "cstddef"
#include <iostream>
#include <cstdlib>
#include <cstring>

template <typename T>
class CircularBuffer {
private:
    size_t head_;
    size_t rear_;
    size_t size_;
    size_t capacity_;
    T **data_;

public:
    CircularBuffer() : head_(0), rear_(0), size_(0), capacity_(0), data_(nullptr) {
        // fprintf(stderr, "in def CB\n");
    }

    explicit CircularBuffer(size_t count) : head_(0), rear_(0), size_(0), capacity_(count) {
        data_ = new T *[count];
    }
    CircularBuffer(const CircularBuffer<T> &other)
        : head_(other.head_), rear_(other.rear_), size_(other.size_), capacity_(other.capacity_) {
        data_ = new T *[capacity_];
        for (size_t i = 0; i < size_; i++) {
            data_[(i + head_) % capacity_] = other.data_[(i + head_) % capacity_];
        }
    }
    CircularBuffer<T> &operator=(const CircularBuffer<T> &other) {
        if (this != &other) {
            // fprintf(stderr, "in cb copy\n");
            for (size_t i = 0; i < capacity_; i++) {
                delete data_[(head_ + i) % capacity_];
            }
            delete[] data_;
            size_ = other.size_;
            capacity_ = other.capacity_;
            rear_ = other.rear_;
            head_ = other.head_;

            if (capacity_ > 0) {
                data_ = new T *[capacity_];
                for (size_t i = 0; i < size_; i++) {
                    T tmp = *(other.data_[(head_ + i) % capacity_]);
                    data_[(head_ + i) % capacity_] = new T(tmp);
                }
                for (size_t i = size_; i < capacity_; i++) {
                    data_[(head_ + i) % capacity_] = new T;
                }
            } else {
                data_ = nullptr;
            }
        }
        return *this;
    }
    ~CircularBuffer() {
        for (size_t i = 0; i < capacity_; i++) {
            delete data_[(head_ + i) % capacity_];
        }
        delete[] data_;
    }

    void Reserve(size_t new_size) {
        if (new_size > capacity_) {
            capacity_ = capacity_ == 0 ? 1 : capacity_;
            // capacity will change so we need to shift all elements in our data.
            ShiftMem();
            capacity_ = new_size;
            T *tmpv = new T[capacity_];
            for (size_t i = 0; i < size_; i++) {
                tmpv[i] = data_[i];
            }
            delete[] data_;
            data_ = tmpv;
            head_ = 0;
            rear_ = size_;
        }
    }
    void UpdateMem() {
        if (size_ >= capacity_) {
            // fprintf(stderr, "in update\n");
            capacity_ = capacity_ == 0 ? 1 : capacity_;
            ShiftMem();
            capacity_ *= 2;
            T **tmpv = new T *[capacity_];
            for (size_t i = 0; i < size_; i++) {
                // fprintf(stderr, "in uodate cycle\n");
                tmpv[i] = data_[i];
            }
            for (size_t i = size_; i < capacity_; i++) {
                tmpv[i] = new T;
                }
            delete[] data_;
            data_ = tmpv;
            head_ = 0;
            rear_ = size_;
        }
    }
    void ShiftMem() {
        T **buff = new T *[capacity_];
        // fprintf(stderr, " in shift\n");
        for (size_t i = 0; i < size_; i++) {
            // fprintf(stderr, "first cycle in shift\n");
            head_ %= capacity_;
            buff[i] = data_[head_++];
        }
        for (size_t i = 0; i < size_; ++i) {
            // fprintf(stderr, "second cycle in shift\n");
            data_[i] = buff[i];
        }
        delete[] buff;
    }
    void PushBack() {
        // std::cout << "true\n";
        // fprintf(stderr, " before UpdateMem\n");
        UpdateMem();

        rear_ %= capacity_;
        rear_++;
        // data_[rear_++] = new T(elem);
        rear_ %= capacity_;
        ++size_;
        // fprintf(stderr, "%p - ptr in push back\n", &data_[0]);
    }

    void PushFront() {
        UpdateMem();
        // head will move "upside down" while pushing and conversely while popping
        head_ = (head_ + capacity_ - 1) % capacity_;
        // data_[head_] = new T(elem);
        ++size_;
    }

    T &Back() {
        return *(data_[(rear_ + capacity_ - 1) % capacity_]);
    }
    const T &Back() const {
        return *(data_[(rear_ + capacity_ - 1) % capacity_]);
    }
    T &operator[](size_t idx) {
        return *(data_[(head_ + idx) % capacity_]);
    }
    const T &operator[](size_t idx) const {
        return *(data_[(head_ + idx) % capacity_]);
    }
    T &Front() {
        return *(data_[head_ % capacity_]);
    }
    const T &Front() const {
        return *(data_[head_ % capacity_]);
    }

    T PopBack() {
        T tmp = Back();
        --size_;
        // it is like usual popping but so that it dependen from capacity
        rear_ = (rear_ + capacity_ - 1) % capacity_;
        return tmp;
    }

    T PopFront() {
        T tmp = Front();
        size_--;
        head_ = (head_ + 1) % capacity_;
        return tmp;
    }

    void Clear() {
        size_ = 0;
        head_ = 0;
        rear_ = 0;
    }
    bool Empty() const {
        return size_ == 0;
    }
    size_t Capacity() const {
        return capacity_;
    }
    size_t Size() const {
        return size_;
    }
    void Swap(CircularBuffer<T> &other) {
        std::swap(data_, other.data_);
        std::swap(capacity_, other.capacity_);
        std::swap(size_, other.size_);
        std::swap(head_, other.head_);
        std::swap(rear_, other.rear_);
    }
};

#endif  // DEQUE_CIRCULAR_BUFFER_H

In [74]:
%%cpp page.h

#ifndef DEQUE_PAGE_PAGE_H
#define DEQUE_PAGE_PAGE_H

#include "cstddef"
#include <iostream>
#include <cstdlib>
#include <cstring>

template <class T>
class Page {
private:
    size_t head_;
    size_t rear_;
    size_t size_;
    size_t capacity_;
    bool frontal_;
    T *data_;

public:
    Page() : head_(0), rear_(0), size_(0), capacity_(100), frontal_(true) {
        data_ = new T[capacity_];
    }
    bool IsFront() const {
        if (size_ == capacity_) {
            return false;
        }
        return frontal_;
    }
    bool IsBack() const {
        if (size_ == 0) {
            return true;
        }
        if (size_ == capacity_) {
            return false;
        }
        return !frontal_;
    }
    Page(const Page<T> &other)
        : head_(other.head_)
        , rear_(other.rear_)
        , size_(other.size_)
        , capacity_(other.capacity_)
        , frontal_(other.frontal_) {
        data_ = new T[capacity_];
        for (size_t i = 0; i < size_; i++) {
            data_[(i + head_) % capacity_] = other.data_[(i + head_) % capacity_];
        }
    }
    Page<T> &operator=(const Page<T> &other) {
        if (this != &other) {
            // fprintf(stderr, "soidv\n");
            size_ = other.size_;
            capacity_ = other.capacity_;
            frontal_ = other.frontal_;
            head_ = other.head_;
            rear_ = other.rear_;
            delete[] data_;
            if (capacity_ > 0) {
                data_ = new T[capacity_];
                for (size_t i = 0; i < capacity_; i++) {
                    data_[(i + head_) % capacity_] = other.data_[(i + head_) % capacity_];
                }
            } /*else data_ = nullptr;*/
        }
        return *this;
    }
    ~Page() {
        delete[] data_;
    }
    void PushBack(T elem) {
        // UpdateMem();
        if (size_ == 0) {
            frontal_ = false;
        }
        rear_ %= capacity_;

        data_[rear_++] = elem;
        rear_ %= capacity_;
        ++size_;
    }

    void PushFront(T elem) {
        // UpdateMem();
        // head will move "upside down" while pushing and conversely while popping
        head_ = (head_ + capacity_ - 1) % capacity_;
        data_[head_] = elem;
        ++size_;
    }
    T &Back() {
        return data_[(rear_ + capacity_ - 1) % capacity_];
    }
    const T &Back() const {
        return data_[(rear_ + capacity_ - 1) % capacity_];
    }
    T &operator[](size_t idx) {
        return data_[(head_ + idx) % capacity_];
    }
    const T &operator[](size_t idx) const {
        return data_[(head_ + idx) % capacity_];
    }
    T &Front() {
        return data_[head_ % capacity_];
    }
    const T &Front() const {
        return data_[head_ % capacity_];
    }

    T PopBack() {
        T tmp = Back();
        --size_;
        // it is like usual popping but so that it dependen from capacity
        rear_ = (rear_ + capacity_ - 1) % capacity_;
        if (size_ == 0) {
            frontal_ = true;
        }
        return tmp;
    }

    T PopFront() {
        T tmp = Front();
        size_--;
        head_ = (head_ + 1) % capacity_;
        return tmp;
    }

    void Clear() {
        size_ = 0;
        head_ = 0;
        rear_ = 0;
        frontal_ = true;
    }
    bool Empty() const {
        return size_ == 0;
    }
    bool Full() const {
        return size_ == capacity_;
    }
    size_t Capacity() const {
        return capacity_;
    }
    size_t Size() const {
        return size_;
    }
};

#endif  // DEQUE_PAGE_PAGE_H

In [76]:
%%cpp dq.h

#ifndef DEQUE_DEQUE_H
#define DEQUE_DEQUE_H

#include "page.h"
#include "cb.h"

const int kPageSize = 100;

template <class T>
class Deque {
private:
    size_t npages_;

public:
    CircularBuffer<Page<T>> data_;
    Deque() : /*data_(1), npages_(0)*/ npages_(0), data_() {
    }
    Deque(const Deque<T> &other) {
        data_ = other.data_;
        npages_ = other.npages_;
    }
    Deque<T> &operator=(const Deque<T> &other) {
        if (this != &other) {
            data_ = other.data_;
            npages_ = other.npages_;
        }
        return *this;
    }
    ~Deque() = default;
    size_t Size() const {
        if (npages_ == 0) {
            return 0;
        }
        if (npages_ == 1) {
            return data_.Front().Size();
        }
        return data_.Front().Size() + (npages_ - 2) * kPageSize + data_.Back().Size();
    }
    void Swap(Deque<T> &other) {
        data_.Swap(other.data_);
        std::swap(npages_, other.npages_);
    }
    T &operator[](size_t idx) {
        if (data_.Front().Size() > idx) {
            return data_.Front()[idx];
        }

        size_t back_strt_idx = data_.Front().Size() + (npages_ - 2) * kPageSize;
        if (idx >= back_strt_idx) {
            return data_.Back()[idx - back_strt_idx];
        }
        return data_[(idx - data_.Front().Size()) / kPageSize + 1][(idx - data_.Front().Size()) % kPageSize];
    }
    const T &operator[](size_t idx) const {
        if (data_.Front().Size() > idx) {
            return data_.Front()[idx];
        }

        size_t back_strt_idx = data_.Front().Size() + (npages_ - 2) * kPageSize;
        if (idx >= back_strt_idx) {
            return data_.Back()[idx - back_strt_idx];
        }
        return data_[(idx - data_.Front().Size()) / kPageSize + 1][(idx - data_.Front().Size()) % kPageSize];
    }
    void Clear() {
        for (size_t i = 0; i < data_.Size(); i++) {
            data_[i].Clear();
        }
        data_.Clear();
        npages_ = 0;
    }

    void PushFront(T elem) {
        if (data_.Size() == 0) {
            // Page<T> tmp;
            data_.PushFront();
            npages_++;
        } else if (data_.Front().Size() == kPageSize) {
            // Page<T> tmp;
            data_.PushFront();
            npages_++;
        }
        data_.Front().PushFront(elem);
    }
    void PushBack(T elem) {
        if (data_.Size() == 0) {
            // fprintf(stderr, "true\n");
            // Page<T> tmp;
            data_.PushBack();
            npages_++;
        } else if (data_.Back().Size() == kPageSize) {
            // fprintf(stderr, "again pushing page\n");
            // Page<T> tmp;
            data_.PushBack();
            npages_++;
        }
        data_.Back().PushBack(elem);
    }
    T PopFront() {
        if (data_.Front().Size() == 0) {
            data_.PopFront();
            npages_--;
        }
        T tmp = data_.Front().PopFront();
        if (data_.Front().Size() == 0) {
            data_.PopFront();
            npages_--;
        }
        return tmp;
    }
    T PopBack() {
        if (data_.Back().Size() == 0) {
            data_.PopBack();
            npages_--;
        }
        T tmp = data_.Back().PopBack();

        if (data_.Back().Size() == 0) {
            data_.PopBack();
            npages_--;
        }
        return tmp;
    }
};

#endif  // DEQUE_DEQUE_H

Run: `clang++ -Wall -Werror main.cpp -o a.exe`

Run: `time -p ./a.exe`

real 0.84
user 0.79
sys 0.04


In [80]:
%%cpp main.cpp
%run clang++ -Wall -Werror main.cpp -o a.exe
%run time -p ./a.exe 

#include <vector>


int main() {
    std::vector<int> a;
    for (int i = 0; i < 10'000'000; ++i) {
        a.push_back(i);
    }
    return 0;
}

Run: `clang++ -Wall -Werror main.cpp -o a.exe`

Run: `time -p ./a.exe`

real 0.30
user 0.24
sys 0.05


In [81]:
%%cpp main.cpp
%run clang++ -Wall -Werror main.cpp -o a.exe
%run time -p ./a.exe 

#include <deque>


int main() {
    std::deque<int> a;
    for (int i = 0; i < 10'000'000; ++i) {
        a.push_back(i);
    }
    return 0;
}

Run: `clang++ -Wall -Werror main.cpp -o a.exe`

Run: `time -p ./a.exe`

real 0.26
user 0.25
sys 0.00


In [84]:
%%cpp main.cpp
%run clang++ -Wall -Werror main.cpp -o a.exe
%run time -p ./a.exe 

#include "dq.h"


int main() {
    Deque<int> a;
    for (int i = 0; i < 10'000'000; ++i) {
        a.PushBack(i);
    }
    return 0;
}

Run: `clang++ -Wall -Werror main.cpp -o a.exe`

Run: `time -p ./a.exe`

real 0.92
user 0.87
sys 0.04


In [86]:
%%cpp main.cpp
%run clang++ -Wall -Werror main.cpp -o a.exe
%run time -p ./a.exe 

#include "dq.h"


int main() {
    CircularBuffer<int> a;
    for (int i = 0; i < 10'000'000; ++i) {
        a.PushBack();
        a.Back() = i;
    }
    return 0;
}

Run: `clang++ -Wall -Werror main.cpp -o a.exe`

Run: `time -p ./a.exe`

real 3.49
user 2.58
sys 0.83
